In [1]:
# Import the necessary libraries
import spacy
import pandas as pd
from spacytextblob.spacytextblob import SpacyTextBlob

# Import transformers for sequence classification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Import libraries for sentiment analysis report
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [2]:
# Load the spacy model
nlp = spacy.load('en_core_web_sm')

In [3]:
# Add spacytextblob to the pipeline
nlp.add_pipe('spacytextblob')

In [4]:
# Load a pre-trained sarcasm detection model
sarcasm_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-sarcasm-twitter")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
# Create a pre-trained sequence classification model
sarcasm_model = AutoModelForSequenceClassification.from_pretrained("mrm8488/t5-base-finetuned-sarcasm-twitter")

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at mrm8488/t5-base-finetuned-sarcasm-twitter and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Instantiate the pipeline
sarcasm_pipeline = pipeline("text-classification", model=sarcasm_model, tokenizer=sarcasm_tokenizer)

In [7]:
# Load the dataset
df = pd.read_csv('amazon_product_reviews.csv')

In [8]:
# Show the dataset
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,https://www.newegg.com/Product/Product.aspx%25...
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,4,https://reviews.bestbuy.com/3545/5442403/revie...,Didnt know how much i'd use a kindle so went f...,Great for the price,johnnyjojojo,https://www.newegg.com/Product/Product.aspx%25...
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-10-09T00:00:00Z,True,177283626.0,3,5,https://redsky.target.com/groot-domain-api/v1/...,I am 100 happy with my purchase. I caught it o...,A Great Buy,Kdperry,https://www.newegg.com/Product/Product.aspx%25...
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/revie...,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack,https://www.newegg.com/Product/Product.aspx%25...


In [9]:
# Drop duplicate values on 'reviews.text' column
df.drop_duplicates(subset='reviews.text', keep=False, inplace=True)

In [10]:
# Print 'reviews.text' column
print(df['reviews.text'])

0       I thought it would be as big as small paper bu...
1       This kindle is light and easy to use especiall...
3       I am 100 happy with my purchase. I caught it o...
4       Solid entry level Kindle. Great for kids. Gift...
6       I ordered this for my daughter, as I have the ...
                              ...                        
4992    My 6yr old son loves the tablet!I plan on purc...
4993    I APPRECIATE the ease of purchasing and readin...
4995    This is a great tablet for the price. Amazon i...
4996    This tablet is the perfect size and so easy to...
4998    I had some thoughts about getting this for a 5...
Name: reviews.text, Length: 3835, dtype: object


In [11]:
# Preprocess the text
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = [token for token in doc if not token.is_stop and not token.is_punct and not token.is_digit and not token.is_currency]
    # filtered_tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct and not token.is_digit and not token.is_currency]
    lemmatized_tokens = [token.lemma_ for token in filtered_tokens]
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

In [12]:
# Sarcasm Detection
def detect_sarcasm(text):
    result = sarcasm_pipeline(text)
    return result[0]['label']

In [13]:
# Perform sentiment analysis
def sentiment_analysis(text):
    doc = nlp(text)
    polarity = doc._.blob.polarity
    subjectivity = doc._.blob.subjectivity
    sarcasm = detect_sarcasm(text)
    if sarcasm == 'SARCASM':
        polarity *= -1
    if polarity > 0:
        sentiment = 'Positive'
    elif polarity < 0:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'    
    return sentiment, polarity, subjectivity, sarcasm

In [14]:
# Preprocess the 'reviews.text' column
df['preprocessed_data'] = df['reviews.text'].apply(preprocess)

In [15]:
# Analyze the sentiment of the preprocessed text
df['sentiment'], df['polarity'], df['subjectivity'], df['sarcasm'] = zip(*df['preprocessed_data'].apply(sentiment_analysis))

In [16]:
# Check the dataset
df.tail()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,preprocessed_data,sentiment,polarity,subjectivity,sarcasm
4992,AVqkIdZiv8e3D1O-leaJ,2017-03-06T14:59:25Z,2017-09-04T11:19:31Z,"Fire Tablet with Alexa, 7"" Display, 16 GB, Mag...",B018Y224PY,Amazon,"Tablets,Fire Tablets,Electronics,iPad & Tablet...",Electronics,https://images-na.ssl-images-amazon.com/images...,"841667103150,0841667103150,firetabletwithalexa...",...,http://reviews.bestbuy.com/3545/5025900/review...,My 6yr old son loves the tablet!I plan on purc...,Great tablet!,QueenT,"https://www.barcodable.com/upc/841667103150,ht...",6yr old son love tablet!i plan purchase Fire t...,Positive,0.300000,0.400000,LABEL_0
4993,AVqkIdZiv8e3D1O-leaJ,2017-03-06T14:59:25Z,2017-09-04T11:19:31Z,"Fire Tablet with Alexa, 7"" Display, 16 GB, Mag...",B018Y224PY,Amazon,"Tablets,Fire Tablets,Electronics,iPad & Tablet...",Electronics,https://images-na.ssl-images-amazon.com/images...,"841667103150,0841667103150,firetabletwithalexa...",...,http://reviews.bestbuy.com/3545/5025900/review...,I APPRECIATE the ease of purchasing and readin...,Im so happy with this product - Reading is GREAT!,Toddler,"https://www.barcodable.com/upc/841667103150,ht...",appreciate ease purchase reading find book tit...,Positive,0.136364,0.454545,LABEL_0
4995,AVqkIdZiv8e3D1O-leaJ,2017-03-06T14:59:25Z,2017-09-04T11:19:31Z,"Fire Tablet with Alexa, 7"" Display, 16 GB, Mag...",B018Y224PY,Amazon,"Tablets,Fire Tablets,Electronics,iPad & Tablet...",Electronics,https://images-na.ssl-images-amazon.com/images...,"841667103150,0841667103150,firetabletwithalexa...",...,http://reviews.bestbuy.com/3545/5025900/review...,This is a great tablet for the price. Amazon i...,Good product,litle,"https://www.barcodable.com/upc/841667103150,ht...",great tablet price Amazon good job,Positive,0.750000,0.675000,LABEL_0
4996,AVqkIdZiv8e3D1O-leaJ,2017-03-06T14:59:25Z,2017-09-04T11:19:31Z,"Fire Tablet with Alexa, 7"" Display, 16 GB, Mag...",B018Y224PY,Amazon,"Tablets,Fire Tablets,Electronics,iPad & Tablet...",Electronics,https://images-na.ssl-images-amazon.com/images...,"841667103150,0841667103150,firetabletwithalexa...",...,http://reviews.bestbuy.com/3545/5025900/review...,This tablet is the perfect size and so easy to...,Great Tablet,gracie,"https://www.barcodable.com/upc/841667103150,ht...",tablet perfect size easy use read play game pu...,Positive,0.458333,0.745833,LABEL_1
4998,AVqkIdZiv8e3D1O-leaJ,2017-03-06T14:59:25Z,2017-09-04T11:19:31Z,"Fire Tablet with Alexa, 7"" Display, 16 GB, Mag...",B018Y224PY,Amazon,"Tablets,Fire Tablets,Electronics,iPad & Tablet...",Electronics,https://images-na.ssl-images-amazon.com/images...,"841667103150,0841667103150,firetabletwithalexa...",...,http://reviews.bestbuy.com/3545/5025900/review...,I had some thoughts about getting this for a 5...,Very sturdy for a 5 year old,Mrbilly,"https://www.barcodable.com/upc/841667103150,ht...",thought get year old screen protector case fee...,Positive,0.025000,0.300000,LABEL_1


In [17]:
# Show the 5 most negative reviews along with their polarity
for index, row in df.sort_values('polarity').head().iterrows():
    print(f"Review: {row['reviews.text']}")
    print(f"Preprocessed Text: {row['preprocessed_data']}")
    print(f"Polarity: {row['polarity']}")
    print(f"Sarcasm: {row['sarcasm']}")
    print()

Review: I'm disappointed that it doesn't have a mirror display mode.
Preprocessed Text: disappointed mirror display mode
Polarity: -0.75

Review: I had to return my device due to WIFI signal dropping, maybe I just had a bad unit.
Preprocessed Text: return device WIFI signal dropping maybe bad unit
Polarity: -0.6999999999999998

Review: I got this because it is almost impossible to break
Preprocessed Text: get impossible break
Polarity: -0.6666666666666666

Review: Just lock the card out and let the kids go crazy!!
Preprocessed Text: lock card let kid crazy
Polarity: -0.6

Review: It works well for me, however, my wife seems to have cold fingers, and the touch screen does not always respond. One other odd quirk. If the tablet is turned off, and you plug it in to charge it, it automatically turns on. You then have to wait until it boots, and then turn it off again. This is very annoying. Before recommending to someone, I would tell them about this annoying quirk.
Preprocessed Text: work 

In [18]:
# Show the 5 most positive reviews along with their polarity
for index, row in df.sort_values('polarity', ascending=False).head().iterrows():
    print(f"Review: {row['reviews.text']}")
    print(f"Preprocessed Text: {row['preprocessed_data']}")
    print(f"Polarity: {row['polarity']}")
    print(f"Sarcasm: {row['sarcasm']}")
    print()

Review: Better than I expected. Parental control is an awesome feature.
Preprocessed Text: well expect parental control awesome feature
Polarity: 1.0

Review: I purchased this so my daughter could read her online books. Perfect size!
Preprocessed Text: purchase daughter read online book perfect size
Polarity: 1.0

Review: Was looking for tablet to read books and this is perfect for that.
Preprocessed Text: look tablet read book perfect
Polarity: 1.0

Review: No issues perfect works as advertised........................
Preprocessed Text: issue perfect work advertise
Polarity: 1.0

Review: This item is perfect for online reading and internet browsing. The only con is that Amazon doesn't have some of the apps that Google Play has.
Preprocessed Text: item perfect online reading internet browsing con Amazon app Google Play
Polarity: 1.0



In [19]:
# Define a function to create a report
def report():
    r = canvas.Canvas("sentiment_analysis_report.pdf")
    width, height = letter

    r.setFont("Helvetica", 10)
    text = [
        "Sentiment Analysis Report:",
        "1. Description of the dataset:",
        "   - The dataset consists of consumer reviews of Amazon products.",
        "   - Customer reviews are represented as text strings in the 'review.text' column.",
        "2. Preprocessing steps:",
        "   - Stop words, punctuation, digits and currency were removed.",
        "   - Converted text to lower case for consistency.",
        "3. Evaluation of the results:",
        "   - The sentiment analysis was performed on a sample of product reviews.",
        "   - Sentiments were classified as Positive, Negative or Neutral based on polarity scores.",
        "   - Sentiments were classified as Sarcasm based on a pretrained model fitted.",
        "4. Insights into model's strengths and limitations:",
        "   - Model strengths: Increased effectiveness for sentiment analysis.",
        "   - Model limitations: May misunderstand some words (such as break or crazy) as negative in some cases.",
    ]

    for i, line in enumerate(text, start=1):
        r.drawString(30, height - 30*i, line)
    r.save()

In [20]:
# Create the PDF report
report()